# Banc Statique <br>
### STAGE ANCHES <br>
Camille Urban <br>
22/03/2024

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from skimage.io import imread
from skimage.transform import rotate
from im_rotate import im_rotate
from scipy import ndimage, datasets

In [3]:
%matplotlib
%matplotlib

Using matplotlib backend: <object object at 0x00000217FF3A3140>
Using matplotlib backend: TkAgg


## Paramètres du Banc

In [4]:
# SENSIBILITE CAPTEUR
S_force_bs = 29.67   # bs pour banc statique
G_force_bs = 100

# cannal d'acquisition
# Cannal 2 

## Fonctions 

In [5]:
# Arrondi la matrice entière 
def round_data (A, n): # renseigner la matrice à traiter et le nombre de chiffres significatif de l'arrondi
    A_round = np.empty(A.shape)     # matrice vide de même taille que la matrice d'entrée
    for i in range (A.shape[0]):
        for j in range (A.shape[1]):
            A_r = round(A[i,j], n)   # arrondi toute la matrice
            A_round[i,j] = A_r
    return A_round

In [7]:
# Mise en grandeur physique
def grandeur_physique(A, Sensibilite, offset, n):
    A_physique = np.empty(A.shape) 
    for i in range (A.shape[0]):
        A_p = round((A[i,2]-offset)/Sensibilite, n)   # arrondi toute la matrice
        A_physique[i,] = (A[i,0], A[i,1], A_p)
    return A_physique

In [31]:
# EXEMPLE utilisation : 
A1 = pd.read_table(f'Banc statique/2024.04.05/anche1/data_anche1.txt', sep=' ').values
A = A1 
n1 = 2 # ordre de l'arrondi
S1 = 29.67 # N/mV
off1 = A1[0,2]
A2 = grandeur_physique(A1, S1, off1, n1)
A3 = round_data(A2, n1)
print(A3)

[[ 0.2   1.67  0.  ]
 [ 0.2   3.33  0.78]
 [ 0.2   5.    1.57]
 [ 0.2   6.67  2.38]
 [ 0.4   1.67  0.36]
 [ 0.4   3.33  1.38]
 [ 0.4   5.    2.51]
 [ 0.4   6.67  3.51]
 [ 0.6   1.67  0.8 ]
 [ 0.6   3.33  2.12]
 [ 0.6   5.    3.36]
 [ 0.6   6.67  4.67]
 [ 0.8   1.67  0.84]
 [ 0.8   3.33  2.2 ]
 [ 0.8   5.    3.52]
 [ 0.8   6.67  4.85]
 [ 1.    1.67  0.82]
 [ 1.    3.33  2.19]
 [ 1.    5.    3.52]
 [ 1.    6.67  4.78]
 [ 1.2   1.67  0.43]
 [ 1.2   3.33  1.62]
 [ 1.2   5.    2.69]
 [ 1.2   6.67  3.9 ]
 [ 1.4   1.67  0.16]
 [ 1.4   3.33  1.1 ]
 [ 1.4   5.    2.09]
 [ 1.4   6.67  2.99]
 [ 1.6   1.67 -0.21]
 [ 1.6   3.33  0.42]
 [ 1.6   5.    1.01]
 [ 1.6   6.67  1.58]]


## Traitement données 

In [9]:
# # extraction données pour plusieurs fichiers
# data = []
# data1 = []

# S = S_force_bs  #sensibilité du capteur
# arrondi = 2

# for i in range (3):
#     for j in range (3):
#         data1 = pd.read_table(f'Banc statique\\2024.03.21\position{i+1}\\anche{j+1}\data_anche{j+1}.txt', sep=' ', header=0).values
#         data1 = round_data(data1, arrondi)
#         data1 = grandeur_physique(data1, S, arrondi)
#         data.append(data1)

# # A = pd.read_table('Banc statique\\2024.03.21\position1\\anche1\data_anche1.txt', sep=' ').values
# # data.append(A)
        
# # print(np.shape(data))
# # print(type(data[0]))
# # print(data),

In [34]:
# extractionnées de donnéespour un seule fichier

data = []
data1 = []
S = S_force_bs  #sensibilité du capteur
arrondi = 2

data1 = pd.read_table(f'Banc statique/2024.04.05/anche2/data_anche2.txt', sep=' ').values

offset = data1[0,2]
for i in range (len(data1)) :
    data1[i,2]=(data1[i,2]-offset)/S

data1 = np.round(data1, arrondi)
# data1 = grandeur_physique(data1, S, arrondi)
# data.append(data1)
# print()

In [35]:
# graphique 2D de la force appliquée en chaque point de mesure
plt.close('all')
fig = plt.figure()

dossier = "Banc statique/2024.04.05/"
savename = "pentes"
palette_couleurs = plt.cm.viridis

data_a = data1
pentes = []

for i in range (0, len(data_a), 4):
    # palette de couleurs
    couleur = palette_couleurs(i / len(data_a))
    
    # trace les courbes de la force mesurées en fonction de l'appuie pour chaque position sur la largeur d'anche
    plt.plot(data_a[i:i+4,1], data_a[i:i+4,2], marker='*', color=couleur, label=f'position d\'appui {data_a[i,0]*10} mm')
    
    # extractions des paramètres de la courbe paramétrique
    coefficients = np.polyfit(data_a[i:i+4,1], data_a[i:i+4,2], 1)
    pente = coefficients[0]
    pente = round(pente, 2)
    ordonnee_origine = coefficients[1]
    ordonnee_origine = round(ordonnee_origine, 2)
    pentes.append(pente)
    
    # trace la courbe paramétrique
    # plt.plot(x=data_a[i:i+4,1], np.polyval(coefficients, data_a[i:i+4,1]), label=f'Droite de régression (pente={pente:.2f}, ordonnée origine={ordonnee_origine:.2f})')
    plt.plot(data_a[i:i+4,1], np.polyval(coefficients, data_a[i:i+4,1]), linestyle='--', color=couleur, label=f'pente={pente:.2f}')
    # print(f'pente n°{i} = {pente}')
  
# enregister en .txt
# pentes = np.array(pentes)  
# pentes = round_data(pentes, 2)

# np.savetxt(dossier + savename, 'pentes')
    
# Étiquetage des axes 2D
plt.title('Force mesurée en fonction de l\'appui vertical (position en z) pour chaque \n position y sur la largeur d\'anche', fontsize=16)
plt.xlabel('Coordonnée z (x 0.1 mm)', fontsize=16)
plt.ylabel('Force F', fontsize=16)
# plt.legend(title= 'position y (mm)') 
plt.legend()

In [38]:
# graphique 3D de la force appliquée en chaque point de mesure
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

data_a = data1
# print(data_a)
for i in range (0, len(data_a), 4):
    # print(len(data_a))
    ax.scatter(data_a[i:i+4,0]*10, data_a[i:i+4,1]*0.1, data_a[i:i+4,2], c='r', marker='o')


# Étiquetage des axes 3D
ax.set_xlabel('Coordonnée y (mm)')
ax.set_ylabel('Coordonnée z (mm)')
ax.set_zlabel('Force F (N)')

Text(0.5, 0, 'Force F (N)')

In [13]:
# Régression linéaire


## Traitement images

In [34]:
# Charger l'image
dossier = "Banc statique/2024.04.05/anche1/"
image = cv2.imread(dossier + 'image_0_0.jpg')  # Remplacez 'votre_image.jpg' par le chemin de votre propre image

# Convertir l'image en niveaux de gris
image_grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Appliquer un flou gaussien pour réduire le bruit
image_blurred = cv2.GaussianBlur(image_grayscale, (5, 5), 0)

# Détection de contours avec l'algorithme de Canny
# contours = cv2.Canny(image_blurred, 50, 150)  # Les valeurs 50 et 150 sont des seuils min et max

# Appliquer un seuillage à l'image (si nécessaire)
_, thresholded_image = cv2.threshold(image_grayscale, 127, 255, cv2.THRESH_BINARY)

# Trouver les contours dans l'image seuillée
contours, _ = cv2.findContours(thresholded_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Dessiner les contours trouvés sur l'image d'origine
contoured_image = cv2.drawContours(image.copy(), contours, -1, (0, 255, 0), 2)

# Afficher l'image originale et les contours détectés
plt.figure(figsize=(12, 6))

# plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title('Image originale')
plt.axis('off')

plt.subplot(1, 2, 2)
cv2.imshow('Contours', contoured_image)
# plt.imshow(contours, cmap='gray')
plt.title('Contours détectés')
plt.axis('off')

plt.show()


In [15]:
plt.close('all')
# Charger l'image
dossier = "Banc statique/2024.04.05/anche1/"
image = cv2.imread(dossier + 'image_0_0.jpg')  # Remplacez 'votre_image.jpg' par le chemin de votre propre image

# Convertir l'image en niveaux de gris
image_grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Appliquer le seuillage pour obtenir une image binaire (noir et blanc)
_, binary_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

# Appliquer un flou gaussien pour réduire le bruit
image_blurred = cv2.GaussianBlur(image_grayscale, (5, 5), 0)

# Détection de contours avec l'algorithme de Canny
contours = cv2.Canny(image_blurred, 30, 90)  # Les valeurs 50 et 150 sont des seuils min et max

# Afficher l'image originale et les contours détectés
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title('Image originale')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(binary_image, cmap='gray')
plt.title('Contours détectés')
plt.axis('off')

plt.show()


In [16]:
plt.close('all')

# Charger l'image
dossier = "Banc statique/2024.04.05/anche1/"
image = plt.imread(dossier + 'image_0_0.jpg')  # Remplacez 'votre_image.jpg' par le chemin de votre propre image
# Convertir l'image en niveau de gris
gray_image = np.mean(image, axis=2)
# Appliquer le seuillage pour obtenir une image binaire (noir et blanc)
binary_image = np.where(gray_image > 127, 255, 0)

# Afficher l'imag
plt.imshow(binary_image, cmap='gray')
plt.title('Cliquez deux fois pour sélectionner des points')
plt.axis('image')

# Demander à l'utilisateur de sélectionner deux points en cliquant sur l'image
points = plt.ginput(2, timeout=-1)

# Afficher les coordonnées des points sélectionnés
print("Coordonnées du premier point:", points[0])
print("Coordonnées du deuxième point:", points[1])

# Afficher les points sélectionnés sur l'image
plt.plot(points[0][0], points[0][1], 'ro')  # Premier point en rouge
plt.plot(points[1][0], points[1][1], 'ro')  # Deuxième point en rouge

plt.close('all')
points = np.round(points).astype(int)

# Récupérer les coordonnées des points de rognage
x1, y1 = points[0]
x2, y2 = points[1]

# Rogner l'image verticalement entre les deux points sélectionnés
image_cropped = image[y1-100:y2+100, :]

# Afficher l'image rognée
plt.imshow(image_cropped)
plt.title('Image rognée')
plt.axis('image')

plt.show()

In [ ]:
### Mesures Banc Statique
plt.close('all')

dossier = "Banc statique/2024.04.05/anche1/"
nom_anche = 'image_'

I = imread(dossier + nom_anche + '0_0.jpg')
angle_rot = im_rotate(I)


cibles = ['Anche Gauche', 'Anche Droite', 'point d\'appuie du capteur de force']
hmax = 650  # hauteur de l'image

# I = im_rotate(I, -np.degrees(angle_rot), mode='constant', cval=np.mean(I))
I = np.flipud(I)
I = I[370:hmax, :]
W = I.shape[1]
H = I.shape[0]
X = np.arange(W)
Y = np.arange(H)
a = 30
N = 20  # taille fenêtre glissante (en px)
dist_px = 5
Ncibles = len(cibles)
COLORS = plt.cm.cool(np.linspace(0, 1, Ncibles))

Xcibles = np.full(Ncibles, np.nan)
Ycibles = np.full(Ncibles, np.nan)

for i in range(Ncibles + 1):
    plt.figure()
    plt.clf()
    plt.imshow(I, cmap='gray', extent=[0, W, H, 0])
    plt.axis('equal')
    plt.xlim(0, W)
    plt.ylim(0, H)
    plt.axhline(H, color='r')
    

for j in range(i):
    plt.plot(Xcibles[j] + np.array([-1, 1, 1, -1, -1]) * a,
                Ycibles[j] + np.array([-1, -1, 1, 1, -1]) * a,
                color=COLORS[j])  # Utilisation de la colormap cool pour représenter les cibles
    plt.plot(Xcibles[j], Ycibles[j], '+', color=COLORS[j])

if i == Ncibles:
    break
    


#############################################################################################################################################
    # for R in range(1,3):
    #     I = imread(dossier + nom_anche + str(R) + '_0.bmp')
    #     print(np.shape(np.mean(I)))
    #     I = im_rotate(I, -np.degrees(angle_rot), mode='constant', cval=np.mean(I))
    #     I = np.flipud(I)

    #     I = I[150:hmax, :]  # couper l'image pour ne garder qu'une partie utile

    #     W = I.shape[1]  # Width
    #     H = I.shape[0]  # Height

    #     X = np.arange(W)  # coord. non calibrées
    #     Y = np.arange(H)
    #     a = 30
    #     N = 20  # taille fenêtre glissante (en px)
    #     dist_px = 5

    #     # Trouver les bords de l'anche, on gardera les indices pour toutes les anches mesurées de la série
    #     Ncibles = len(cibles)
    #     COLORS = plt.cm.cool(np.linspace(0, 1, Ncibles))

    #     Xcibles = np.full(Ncibles, np.nan)
    #     Ycibles = np.full(Ncibles, np.nan)

    #     for i in range(Ncibles + 1):
    #         plt.figure()
    #         plt.clf()
    #         plt.imshow(I, cmap='gray', extent=[0, W, H, 0])
    #         plt.axis('equal')
    #         plt.xlim(0, W)
    #         plt.ylim(0, H)
    #         plt.axhline(H, color='r')

    #         for j in range(i):
            #     plt.plot(Xcibles[j] + np.array([-1, 1, 1, -1, -1]) * a,
            #              Ycibles[j] + np.array([-1, -1, 1, 1, -1]) * a,
            #              color=COLORS[j])  # Utilisation de la colormap cool pour représenter les cibles
            #     plt.plot(Xcibles[j], Ycibles[j], '+', color=COLORS[j])

            # if i == Ncibles:
            #     break

        #     plt.title('Cliquer sur 1 = Anche Gauche, 2 = Anche Droite -> {} ({})'.format(i + 1, cibles[i]))
        #     xclic, yclic = plt.ginput(1)[0]
        #     c1 = np.array([round(xclic), round(yclic)])  # prendre des entiers
        #     plt.title('')

        #     Xcibles[i] = c1[0]  # pixel x des cibles
        #     Ycibles[i] = c1[1]  # pixel y des cibles

        # Xv = np.arange(Xcibles[0], Xcibles[1])

        # # Detection du profil pour toutes les forces d'une anche
        # for F in range(30):
        #     I = imread(dossier + nom_anche + str(R) + '_' + str(F) + '.bmp')
            # I = im_rotate(I, np.degrees(angle_rot), mode='constant', cval=np.mean(I))
            # I = np.flipud(I)
            # I = I[150:hmax, :]

            # Yv_m = np.zeros_like(Xv)
            # Yv_r = np.zeros_like(Xv)

            # for i in range(len(Xv)):
                # Iv = I[:, Xv[i]]  # vecteur qui contient une tranche de l'image largeur 1 (x) et hauteur de l'image (y)
                # Iv = Iv.astype(float)

                # Iv_2 = np.diff(Iv)  # dérivée

                # smin = -13
                # smax = 10
            #     pos1 = np.where(Iv_2 < smin)[0]
            #     pos2 = np.where(Iv_2 > smax)[0]

            #     if len(pos2) > 0:
            #         Yv_m[i] = Y[pos2[-1]]
            #     else:
            #         Yv_m[i] = np.nan  # peu probable parce que detectera toujours la frontière lèvre/anche

            #     if len(pos1) > 0:
            #         Yv_r[i] = Y[pos1[-1]]
            #     else:
            #         Yv_r[i] = Yv_m[i]  # lorsque le canal d'anche est fermé, il n'y aura pas de Yv_r.

            # plt.plot(Xv, Yv_r, '.-', linewidth=0.1)
            # plt.plot(Xv, Yv_m, 'm.-', linewidth=0.1)
            # plt.pause(0.2)

            # # Calcule de la surface du canal d'anche
            # pix_canal = Yv_m - Yv_r

            # dist_px = 1064  # mesuré sur l'image MIRE 05-03-42 (8 carreaux)
            # dist_mm = 12  # mesuré à la règle sur la mire (cf mon cahier de labo, 2 carreaux = 3mm)
            # S = dist_mm / dist_px  # "sensibilité" (en mm/px)

            # nb_pix_canal = np.sum(pix_canal)  # je compte tous les pixels qu'il y a dans le canal
        #     surf_pix = S ** 2  # surface d'un pixel
        #     dim_canal = nb_pix_canal * surf_pix  # surface en mm² du canal

        #     print('Surface canal estime =', dim_canal, 'mm²')

        # savename = nom_anche + str(R) + '.mat'
        # np.savetxt(dossier + savename, np.array([Yv_r, Yv_m, dim_canal]).T)
        # plt.close('all')




In [ ]:
# Appel de la fonction
dossier = "Banc statique/2024.04.05/anche1/"
nom_anche = 'image_'
# imread(dossier + nom_anche + '0_0.jpg')
detect_profil(dossier, nom_anche)